## Cellpose Pipeline - Cell Size & Foci Analysis

In [ ]:
""" Imports and Functions """
import numpy as np # array (img) analysis
import pandas as pd # working with dtaframes
from cellpose import models # segmentation + DL
from scipy.stats import pearsonr # statistics for relationship (custom)
import seaborn as sns # visualization/plots
import matplotlib.pyplot as plt #  viasualization/plots
from skimage import io, measure, morphology, filters # io -> read image | measure -> extract features | morphology -> clean masks | filters -> threshold and smooth
from scipy import ndimage as ndi# Laplacian of Gaussian (LoG) to detect foci 

In [ ]:
# open and read image
image = io.imread("/Users/jiami/Desktop/w39_sort_merged/MAX_W39F0004T0011_merge1.tif")
# need to double check im extension (tiff or nd2?) + how to work with nd2 files

print("image shape: \n", image.shape)
print("before selection of channel: \n", image)
# Ensure single channel
if image.ndim == 3:
    # take the first channel (if grayscale)
    image = image[:, :, 0]
 
image = image.astype(np.float32) # intensities as float 32 (better precision)
print("image shape: \n", image.shape)
print("after selection of channel: \n", image)

OSError: Could not find a backend to open `/Users/jiami/Downloads/09.26-0%-10%/20250925_50ms30G30R_0%_0.658_0hrs_014.nd2`` with iomode `r`.

In [10]:
# cell segmentation with cellpose
model = models.Cellpose(model_type='cyto')

masks, flows, styles, diams = model.eval(
    image,
    diameter=None,
    channels=[0, 0]
)

/opt/anaconda3/envs/cellpose_env/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/cellpose_env/lib/python3.10/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [11]:
print("Image shape:", image.shape)
print("Masks shape:", masks.shape)
print("Image ndim:", image.ndim)
print("Masks ndim:", masks.ndim)

Image shape: (2, 2000)
Masks shape: (2000,)
Image ndim: 2
Masks ndim: 1


In [12]:
# extract features from the segmented cells
cell_props = measure.regionprops(masks, intensity_image=image)

cell_data = []

for prop in cell_props:
    cell_data.append({
        "cell_id": prop.label,
        "area": prop.area,
        "major_axis_length": prop.major_axis_length,
        "minor_axis_length": prop.minor_axis_length,
        "eccentricity": prop.eccentricity,
        "mean_intensity": prop.mean_intensity
    })

cell_df = pd.DataFrame(cell_data)

TypeError: Only 2-D and 3-D images supported.

In [ ]:
# detecting the foci inside of the green channel

# Smooth image
smoothed = filters.gaussian(image, sigma=1)

# Enhance puncta
log_image = -ndi.gaussian_laplace(smoothed, sigma=1)

# Normalize
log_image = (log_image - log_image.min()) / (log_image.max() - log_image.min())

# Threshold
threshold = filters.threshold_otsu(log_image)
foci_mask = log_image > threshold

# Remove tiny noise
foci_mask = morphology.remove_small_objects(foci_mask, min_size=5)

In [ ]:
# label the foci
foci_labels = measure.label(foci_mask)

In [ ]:
# relate foci to cells
foci_props = measure.regionprops(foci_labels, intensity_image=image)

# Initialize foci metrics per cell
cell_df["foci_count"] = 0
cell_df["foci_total_area"] = 0
cell_df["foci_total_intensity"] = 0

for foci in foci_props:
    coords = foci.coords
    cell_labels = masks[coords[:,0], coords[:,1]]
    
    # Find dominant cell label
    cell_label = np.bincount(cell_labels).argmax()
    
    if cell_label == 0:
        continue
    
    idx = cell_df.index[cell_df["cell_id"] == cell_label][0]
    
    cell_df.loc[idx, "foci_count"] += 1
    cell_df.loc[idx, "foci_total_area"] += foci.area
    cell_df.loc[idx, "foci_total_intensity"] += foci.mean_intensity * foci.area

In [ ]:
# normalize the foci
cell_df["foci_per_area"] = cell_df["foci_count"] / cell_df["area"]
cell_df["foci_area_fraction"] = cell_df["foci_total_area"] / cell_df["area"]

In [ ]:
# Visualize the relationship between cell size and foci count
plt.figure()
sns.scatterplot(
    data=cell_df,
    x="major_axis_length",
    y="foci_count"
)
plt.title("Cell Length vs Foci Count")
plt.show()

In [ ]:
# correlation calculation
r, p = pearsonr(cell_df["major_axis_length"], cell_df["foci_count"])
print("Pearson r:", r)
print("p-value:", p)

In [ ]:
# this is for push